# Лабораторная работа №3

## Задание

Создать сеть на базе LSTM используя TensorFlow (Keras). Сеть должна принимать на вход текстовый файл и на его базе генерировать свою абракадабру. Отчет должен содержать кроме кода, обучающий файл и результат генерации.

## Реализация

### Датасет

В качестве датасета возьмём произведение Моби Дик